---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
# means
mn1 = 0.08
mn2 = 0.12
mn3 = 0.15

# std devs
sd1 = 0.15
sd2 = 0.25
sd3 = 0.35

# correlations
c12 = 0.15
c13 = 0.60
c23 = 0.30

# risk-free rate
rf = 0.02

# example portfolio
w1 = 0.1
w2 = 0.8
w3 = 0.1

# CREATE ARRAYS

In [2]:
import numpy as np 

mn = np.array([mn1, mn2, mn3])
sd = np.array([sd1, sd2, sd3])

corr = np.identity(3)
corr[0, 1] = corr[1, 0] = c12
corr[0, 2] = corr[2, 0] = c13
corr[1, 2] = corr[2, 1] = c23

cov = np.diag(sd) @ corr @ np.diag(sd)

# CHECK CORRELATIONS

In [3]:
if np.all(np.linalg.eigvals(cov) > 0):
    print("correlations are acceptable")
else:
    print("correlations are inconsistent")

correlations are acceptable


# CALCULATE EXAMPLE MEAN AND RISK

In [4]:
example = np.array([w1, w2, w3])
example_mn = example @ mn
example_sd = np.sqrt(example @ cov @ example)
example_sharpe = (example_mn - rf) / example_sd

# CALCULATE FRONTIER

In [5]:
# global minimum variance portfolio
w = np.linalg.solve(cov, np.ones(3))
gmv = w / np.sum(w)
gmv_mn = gmv @ mn
gmv_sd = np.sqrt(gmv @ cov @ gmv)

# second frontier portfolio
w = np.linalg.solve(cov, mn)
second_port = w / np.sum(w)
second_mn = second_port @ mn 
second_sd = np.sqrt(second_port @ cov @ second_port)

# means to display
min_mn = 0
max_mn = 1.2*np.max(mn)
mns = np.linspace(min_mn, max_mn, 101)

# portfolio weights and risks
gmv_wt = (mns - second_mn) / (gmv_mn - second_mn)
second_wt = 1 - gmv_wt
ports = (
    gmv_wt.reshape(-1, 1) * gmv.reshape(1, -1) + 
    second_wt.reshape(-1, 1) * second_port.reshape(1, -1)
)
vr = np.diag(ports @ cov @ ports.T)
sds = np.sqrt(vr)


# FIGURE

In [7]:
import plotly.graph_objects as go 
import plotly.io as pio
plotly_template = pio.templates["simple_white"]
colors = plotly_template.layout.colorway

string = 'asset 1: %{customdata[0]:.0%}<br>'
string += 'asset 2: %{customdata[1]:.0%}<br>'
string += 'asset 3: %{customdata[2]:.0%}<br>'
string += '<extra></extra>'
trace1 = go.Scatter(
    x=sds,
    y=mns,
    mode="lines",
    customdata=ports,
    hovertemplate=string
)

trace2 = go.Scatter( 
    x=sd,
    y=mn,
    text=["Asset 1", "Asset 2", "Asset 3"],
    hovertemplate="%{text}<extra></extra>",
    mode="markers",
    marker=dict(size=15)
)
string = 'example portfolio<br>'
string += 'asset 1: %{customdata[0]:.0%}<br>'
string += 'asset 2: %{customdata[1]:.0%}<br>'
string += 'asset 3: %{customdata[2]:.0%}<br>'
string += '<extra></extra>'
trace3 = go.Scatter(
    x=[example_sd],
    y=[example_mn],
    customdata=example.reshape(1, -1),
    hovertemplate=string,
    mode="markers",
    marker=dict(size=15)
)

x = np.arange(0, 2.01, 0.01) * example_sd 
y = rf + np.arange(0, 2.01, 0.01) * (example_mn - rf)
trace4 = go.Scatter(
    x=x,
    y=y,
    text = [example_sharpe]*len(x),
    mode="lines",
    hovertemplate="Sharpe ratio = %{text:.1%}<extra></extra>",
    line=dict(color=colors[5])
)

fig = go.Figure()
for trace in (trace1, trace2, trace3, trace4):
    fig.add_trace(trace)

fig.update_layout(
    xaxis_title="Standard Deviation",
    yaxis_title="Expected Return",
    xaxis_rangemode = "tozero",
    yaxis_rangemode="tozero",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
    showlegend=False
) 
fig.show()